In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.optim as optim
import torchvision
import torch
torch.set_num_threads(4)

In [2]:
x= np.load( 'training_data.npy')
y=np.load( 'training_labels.npy')
test_data_x=np.load( 'test_data.npy')
test_labels_y=np.load( 'test_labels.npy')
class_labels= np.array([0]*82858+[1]*7354+[2]*480)

In [3]:
import torch
from torch.utils.data import DataLoader, TensorDataset
# create Tensor datasets
train_data = TensorDataset(torch.from_numpy(x), torch.from_numpy(y), torch.from_numpy(class_labels))
test_data = TensorDataset(torch.from_numpy(test_data_x), torch.from_numpy(test_labels_y))
# dataloaders
batch_size = 64
# make sure to SHUFFLE your data
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size, num_workers=4)
test_loader = DataLoader(test_data, shuffle=True, batch_size=batch_size, num_workers=4)
ngpu=1
device = torch.device("cuda:2" if (torch.cuda.is_available() and ngpu > 0) else "cpu")

In [4]:
import torch.nn as nn

class SentimentLSTM(nn.Module):
    """
    The RNN model that will be used to perform Sentiment analysis.
    """

    def __init__(self, output_size, input_dim, hidden_dim, n_layers, drop_prob=0.15):
        """
        Initialize the model by setting up the layers.
        """
        super().__init__()

        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        
        self.lstm = nn.LSTM(input_dim, hidden_dim, n_layers, 
                            dropout=drop_prob, batch_first=False)
        
        # dropout layer
        self.dropout = nn.Dropout(0.15)
        
        # linear and sigmoid layers
        self.fc = nn.Linear(hidden_dim, output_size)
        self.sig = nn.Sigmoid()
        

    def forward(self, x, hidden):
        """
        Perform a forward pass of our model on some input and hidden state.
        """
        batch_size = x.size(1)

        hidden=self.init_hidden(batch_size)
        hidden = tuple([each.data for each in hidden])
        lstm_out, hidden = self.lstm(x, hidden)
    
        # stack up lstm outputs
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)
        
        # dropout and fully-connected layer
        out = self.dropout(lstm_out)
        out = self.fc(out)
        # sigmoid function
        sig_out = self.sig(out)
        
        # reshape to be batch_size first
        sig_out = sig_out.view(batch_size, -1)
        sig_out = sig_out[:, -1] # get last batch of labels
        
        # return last sigmoid output and hidden state
        return sig_out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x hidden_dim,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device),
                  weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device))
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        
        return hidden

In [5]:
class Discriminator (nn.Module ):
    def __init__(self):
        super(Discriminator, self ).__init__()

        outdim = 2440
        self.maxpool = nn.MaxPool1d(kernel_size=2, stride=2)
        self.leakyReLU = nn.LeakyReLU()
        self.conv_1 = nn.Conv1d(1, 3, kernel_size=27, stride=1)
        self.conv_2 = nn.Conv1d(3, 10, kernel_size=14, stride=1)
        self.conv_3 = nn.Conv1d(10, 10, kernel_size=3, stride=1)
        self.conv_4 = nn.Conv1d(10, 10, kernel_size=4, stride=1)
        
        
        self.fc1 = nn.Linear(outdim,30)
        self.fc2 = nn.Linear(30,10)
        self.fc3 = nn.Linear(10,3)

        torch.nn.init.xavier_uniform_(self.conv_1.weight)
        torch.nn.init.xavier_uniform_(self.conv_2.weight)
        torch.nn.init.xavier_uniform_(self.conv_3.weight)
        torch.nn.init.xavier_uniform_(self.conv_4.weight)
        torch.nn.init.xavier_uniform_(self.fc1.weight)
        torch.nn.init.xavier_uniform_(self.fc2.weight)

    def forward(self, x ):
        # 1
        x = self.maxpool(self.leakyReLU(self.conv_1(x ) ) )
        # 3
        x = self.maxpool(self.leakyReLU(self.conv_2(x ) ) )
        # 5
        x = self.maxpool(self.leakyReLU(self.conv_3(x ) ) )
        # 7
        x = self.maxpool(self.leakyReLU(self.conv_4(x ) ) )
        
        x = x.view(x.size(0), -1)
        
        # 9        
        x = self.leakyReLU(self.fc1(x ) )
        # 10
        x = self.leakyReLU(self.fc2(x ) )
    
        x = self.fc3(x )

        return x

In [6]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)
# Create the Discriminator
cnn = Discriminator().to(device)

# Handle multi-gpu if desired
if (device.type == 'cuda') and (ngpu > 1):
    cnn = nn.DataParallel(cnn, list(range(ngpu)))

# Apply the weights_init function to randomly initialize all weights
#  to mean=0, stdev=0.2.
cnn.apply(weights_init)

# Print the model
print(cnn)

Discriminator(
  (maxpool): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (leakyReLU): LeakyReLU(negative_slope=0.01)
  (conv_1): Conv1d(1, 3, kernel_size=(27,), stride=(1,))
  (conv_2): Conv1d(3, 10, kernel_size=(14,), stride=(1,))
  (conv_3): Conv1d(10, 10, kernel_size=(3,), stride=(1,))
  (conv_4): Conv1d(10, 10, kernel_size=(4,), stride=(1,))
  (fc1): Linear(in_features=2440, out_features=30, bias=True)
  (fc2): Linear(in_features=30, out_features=10, bias=True)
  (fc3): Linear(in_features=10, out_features=3, bias=True)
)


In [7]:
# Instantiate the model w/ hyperparams
output_size = 1
input_dim = 1
hidden_dim = 256
n_layers = 2
net = SentimentLSTM(output_size, input_dim, hidden_dim, n_layers)
#net.load_state_dict(torch.load("lstm_adam",map_location="cuda:2"))
net.load_state_dict(torch.load("lstm10",map_location="cpu"))
device = torch.device("cuda:2" if (torch.cuda.is_available() and 1 > 0) else "cpu")
net.to(device)
#print(net)

SentimentLSTM(
  (lstm): LSTM(1, 256, num_layers=2, dropout=0.15)
  (dropout): Dropout(p=0.15)
  (fc): Linear(in_features=256, out_features=1, bias=True)
  (sig): Sigmoid()
)

In [8]:
epsilon=1e-20
def accuracy(outputs, labels):
    temp=(outputs==labels)
    return temp.sum()/ (labels.shape[0])

def true_positive(outputs, labels):
    temp=outputs+labels
    return (temp==2).sum()
    
def true_negative(outputs, labels):
    temp=outputs+labels
    return (temp==0).sum()

def false_positive(outputs, labels):
    temp= labels-outputs
    return (temp<0).sum()

def false_negative(outputs, labels):
    temp=outputs-labels
    return (temp<0).sum()

def precision(outputs, labels):
    return true_positive(outputs, labels)/(true_positive(outputs, labels)+false_positive(outputs, labels))

def recall(outputs, labels):
    return true_positive(outputs, labels)/(true_positive(outputs, labels)+false_negative(outputs, labels))

def BCR(outputs, labels):
    return (precision(outputs, labels)+recall(outputs, labels))/2

In [15]:
def predict(net, test_seq, sequence_length):
    
    net.eval()

    # pad tokenized sequence
    seq_length=sequence_length
    
    # convert to tensor to pass into your model
    #feature_tensor = torch.from_numpy(test_seq)
    feature_tensor=test_seq.view(4000,sequence_length, 1)
    
    batch_size = feature_tensor.size(0)
    
    # initialize hidden state
    h = net.init_hidden(batch_size)
    h = tuple([each.data for each in h])
    
    
    # get the output from the model
    feature_tensor = feature_tensor.type(torch.FloatTensor)
    feature_tensor = feature_tensor.to(device)
    output, h = net(feature_tensor, h)
    
    # convert output probabilities to predicted class (0 or 1)
    pred = torch.round(output.squeeze()) 
    # printing output value, before rounding
   # print('Prediction value, pre-rounding: {:.6f}'.format(output.item()))
    
    # print custom response
  #  if(pred.item()==1):
  #      print("AFIB wave")
 #   else:
   #     print("Normal wave")
    
    return pred.detach().cpu().numpy()

In [ ]:
plot_train=[];plot_val=[]
# loss and optimization functions
lr=0.001

criterion = nn.BCELoss().to(device)
c1= nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(net.parameters(), lr=lr)
optimizerD = optim.Adam(cnn.parameters(), lr=lr, betas=(0.5, 0.999))
train_on_gpu=True

# training params

epochs = 50 # 3-4 is approx where I noticed the validation loss stop decreasing

counter = 0
print_every = 1000
clip=5 # gradient clipping

# move model to GPU, if available
if(train_on_gpu):
    net.to(device)
    cnn.to(device)

net.train()
m = torch.nn.Sigmoid()
# train for some number of epochs
for e in range(epochs):
    # initialize hidden state
    #h = net.init_hidden(batch_size)

    # batch loop
    train_loss=[];discr_err=[]
    h=None
    for inputs, labels, c_labels in train_loader:
        
        counter += 1
        # Creating new variables for the hidden state, otherwise
        # we'd backprop through the entire training history
        #h = tuple([each.data for each in h])
        
        bs= inputs.shape[0]
        inputs=inputs.view(4000, bs, 1)
        inputs = inputs.type(torch.FloatTensor)
        #if(train_on_gpu):
        inputs, labels, c_labels = inputs.to(device), labels.to(device), c_labels.to(device)
        cnn.zero_grad()
        output = cnn(inputs.view(bs ,1, 4000))
        out=m(output)
        err = c1(out.squeeze(), c_labels)
        err.backward()
        optimizerD.step()
        
        # zero accumulated gradients
        net.zero_grad()
        # get the output from the model
        output, h = net(inputs, h)
        # calculate the loss and perform backprop
        loss = criterion(output.squeeze(), labels.float())-err.item()
        loss.backward()
        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        nn.utils.clip_grad_norm_(net.parameters(), clip)
        optimizer.step()
        train_loss.append(loss.item()); discr_err.append(err.item())

    pred=np.array([]); true_l=np.array([])
    net.eval()
    for inputs, labels in test_loader:
        pred=np.append(pred,predict(net, inputs, inputs.shape[0]), axis=0)
        true_l=np.append(true_l,labels.detach().cpu().numpy(), axis=0)
    test_acc= accuracy(pred, true_l)
    test_prec= precision(pred, true_l)
    test_recall= recall(pred, true_l)
    test_f1= 2*test_prec*test_recall/(test_prec+test_recall)
    print("Accuracy="+ str(test_acc)+"....."+"Precision="+ str(test_prec)+'.....'+"Recall="+ str(test_recall)+'....'+"F1 score="+ str(test_f1))
    net.train()
    print("Epoch: {}/{}...".format(e+1, epochs),
                  "Step: {}...".format(counter),
                  "Tr Loss: {:.6f}...".format(np.mean(train_loss)), "Discr Loss: {:.6f}...".format(np.mean(discr_err)))

    torch.save(net.state_dict(), "adv_lstm"+ str(e))
    torch.save(cnn, "adv_cnn")
    plot_train.append(np.mean(train_loss))
    plot_val.append(test_f1)
np.save('training_lstm', plot_train)
np.save('Validation_lstm', plot_val)

Accuracy=0.5612040133779265.....Precision=0.5632716049382716.....Recall=0.10193632470675852....F1 score=0.1726312470439855
Epoch: 1/50... Step: 1418... Tr Loss: 0.026032... Discr Loss: 0.611155...


/eerun/Python-3.7.2/lib/python3.7/site-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type Discriminator. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [17]:
plot_val

[0.1726312470439855,
 0.2585225171929306,
 0.33956629932456456,
 0.42681593000064344,
 0.4792587658208684,
 0.4267794922847188,
 0.44217099581478775,
 0.6186476716989157,
 0.5030558880977883,
 0.6154172185430463,
 0.6257464817988264,
 0.4791413786494602,
 0.5322009907997169,
 0.4550561797752809,
 0.6849220370228128,
 0.5671113968815303,
 0.6431806264003432,
 0.6438387024104527,
 0.6335795267441313,
 0.6134673366834171,
 0.637060003647638,
 0.6492381236928593,
 0.6409789828041126,
 0.6416429975966791,
 0.6699949716256016,
 0.6566868548356378,
 0.6495770982433312,
 0.6599178761637599,
 0.6504176035978156,
 0.6539438050101557,
 0.6284254528564794,
 0.6835104225809068,
 0.6591675015939521,
 0.6626865671641792,
 0.6507953922106418,
 0.675580997949419,
 0.6623456249452572,
 0.6476994887752834,
 0.6735594277819867,
 0.6564930672979371,
 0.6912257200267917,
 0.6701246621916137,
 0.6916267359701129,
 0.6672462508150402,
 0.6860914900773629,
 0.6562339421684497,
 0.6746958135698082,
 0.669171144